# Compiling a QGL2 InversionRecovery and plotting the output

## Imports

In [ ]:
from pyqgl2.main import compile_function, qgl2_compile_to_hardware
from pyqgl2.test_cl import create_default_channelLibrary
from pyqgl2.qreg import QRegister
from QGL import plot_pulse_files, ChannelLibrary
import numpy as np

### Should sequences be compiled to hardware, or just to QGL?

In [ ]:
toHW = True
# Append suffix to filenames?
suffix = True

### Create a test ChannelLibrary

In [ ]:
create_default_channelLibrary(toHW, True)
# Alternatively could load an existing library, or create one here; see the 'AllXY' notebook

## Create needed qubit(s)

In [ ]:
# For QGL2, use a QRegister, not a QGL Qubit
q = QRegister(1)

## Compile to QGL1

In [ ]:
# Insert proper path to QGL2 source and name of qgl2main if not so marked
# Here we compile the named function in the named file from QGL2 to QGL1 and return the new function

In [ ]:
irDelays = np.linspace(0, 5e-6, 11)
tCalR = 2
qgl1MainFunc = compile_function("../src/python/qgl2/basic_sequences/T1T2.py", "InversionRecovery", (q, irDelays, tCalR))

## Generate pulse sequences

In [ ]:
# Now run the QGL1 function, producing a list of sequences
seqs = qgl1MainFunc()

## Optionally compile to machine instructions

In [ ]:
if toHW:
    from qgl2.basic_sequences.helpers import delay_descriptor, cal_descriptor
    axis_desc = [
            delay_descriptor(irDelays),
            cal_descriptor(('qubit',), tCalR)
            ]
    label = "T1"
    # Generate proper filenames; for these, it isn't just the label T1
    # T1T2 QGL functions take a suffix boolean default false. If true, then append to label "_qubit.label"; ie "_q1"
    if suffix:
        label = label + "_q1"
    
    metaFileName = qgl2_compile_to_hardware(seqs, filename=f"{label}/{label}", axis_descriptor=axis_desc)
    print(f"Generated sequence details in '{metaFileName}'")
    # Plot the sequences
    p = plot_pulse_files(metaFileName)
    # Explicitly display the graph which fails to auto-draw in some cases
    display(p)
else:
    from QGL.Scheduler import schedule
    from IPython.lib.pretty import pretty
    print(pretty(schedule(seqs)))